# Grabbing all meta cards, decklists, and dates

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import random
import time
from datetime import date, datetime, timedelta

In [ ]:
current_date = date.today()
deck_link_list = []
driver = webdriver.Chrome()
for y in range(2019, current_date.year + 1):
    for m in range(1, 13):
        dfrom = datetime(y, m, 1).strftime('%Y-%m-%d')
        dto   = datetime(y, m, 1).strftime('%Y-%m-%d') if m == 12 else (datetime(y, m + 1, 1) + timedelta(days=-1)).strftime('%Y-%m-%d')
        print(f'from {dfrom} to {dto}')

        driver.get(f'https://ygoprodeck.com/deck-search/?&_sft_category=tournament%20meta%20decks&sort=Date&from={dfrom}&to={dto}&offset=0')

        try:
            DECK_LINK_TARGET_CLASS = 'deck_article-card-container'
            RELATIVE_DATE_CLASS = "deck_article-card-stats text-left d-block" #info class
            NEXT_PAGE_TARGET_CLASS = 'prevDeck' #button for next page
            DISABLED_CLASS = 'disabled'
            i = 0
            while(1):
                print('PAGE ', i)

                #need to wait until the page loads the divs containing the links
                elements = WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, DECK_LINK_TARGET_CLASS))
                )

                # Once the content is loaded, you can extract it
                for element in elements:
                    anchor = element.find_element(By.TAG_NAME, 'a')
                    href = anchor.get_attribute('href')
                    print(f'{dfrom}\t{href}')
                    deck_link_list.append((dfrom, href))
                
                #wait for prev button to load
                next_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, NEXT_PAGE_TARGET_CLASS))
                )

                #click prev button if its not disabled
                next_button_classes = next_button.get_attribute('class').split()
                if DISABLED_CLASS in next_button_classes:
                    print('REACHED LAST PAGE, EXITING LOOP')
                    break
                else:
                    time.sleep(random.randint(1, 2))
                    next_button.click()
                
                i+=1
        except TimeoutException as exception:
            print('timeout exception')
            continue
        except:
            print('exception')

driver.quit()

In [ ]:
print(len(deck_link_list))

In [ ]:
import csv

In [ ]:
with open('deck_links.csv', 'w', newline='') as new_file:
    csv_writer = csv.writer(new_file, delimiter=',')
    csv_writer.writerows(deck_link_list)

In [ ]:
# TODO:
# - go through each decklist, and pull all card ids
# - if more accurate date available, pull it and replace the date 
# - pull api json 
# - for each card id, assign any decklist ids it was present in along with the date of the tourney